[문제] Managing Conversation History
- trim_messages()
- RunnablePassthrough 
- itemgetter()

세션ID 설정하고, 대화를 진행합니다. (multi-turn conversation) 
LLM 모델이 과거 대화를 알지(기억) 못하는 상황을 만드세요.

In [14]:
## 1. 모듈(파일, 라이브러리) 읽어오기
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory, BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv


## 2. 환경변수에 설정된 값 읽어오기
load_dotenv()

## 3. 파일 읽기
file_name = '남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률(법률)(제20521호)(20250223).txt'

with open(file_name, 'r', encoding='utf-8') as file:
    law = file.read()

## 4. prompt 생성
template = [
    ('system', '''당신은 육아휴직 법률 전문가입니다.
     아래 문서를 참고하여, 사용자 질문에 성실하게 답변합니다.
     답변은 문서 내용을 기반으로 하되, 해당 조항도 표시합니다.
     - 문서: {law}'''),
     ('placeholder', '{chat_history}'),
    ('user', '{query}')
]

prompt = ChatPromptTemplate.from_messages(template)



In [ ]:

## 5. ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    model='gpt-4o',
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    max_tokens=512, ## 최대 출력
)


from langchain_core.messages import trim_messages


## 6. trim 설정
trimmer = trim_messages(
    max_tokens=65,   ##최대 토큰 수 제한
    strategy='last',  ##최근 메시지부터 시작해서 자르기
    token_counter=llm,  ##모델 기준으로 토큰 수 계산
    include_system=True,  ##시스템 메시지 포함
    allow_partial=False, ##메시지 자르기 제한
    start_on='human', ##human 메시지부터 자르기 시작

)



from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

## 6. chain 구성
chain = (
    RunnablePassthrough.assign(chat_history=itemgetter('chat_history') | trimmer)
    | prompt 
    | llm
    )

## 7. 세션별 대화 이력(히스토리)을 저장할 임시 메모리 저장소
store = {}

## 8. 함수 정의: 대화 이력(히스토리) 인스턴스 리턴
def get_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


## 10. chain에 대화 이력(히스토리) 기능을 래핑해서 추가
with_message_history = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key='chat_history',
    input_messages_key='query',
)


In [22]:
## 11. 실행
while True:
    query = input('육아휴직과 관련된 질문하세요 [종료 S]>>> ')
    if query.upper() == 'S':
        break




    with_message_history.invoke(
        {'law': law, 'query': query},
        config={'configurable': {'session_id':'leeji0422'}}
    ) 

    print('\n'+'='*50+'\n')

남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률에 따르면, 육아휴직과 관련하여 국가에서 지원할 수 있는 주요 지원금은 다음과 같습니다.

1. **육아휴직 급여**: 고용보험에서 지원되는 급여로, 근로자가 육아휴직을 사용하는 경우 경력단절을 방지하고 생활 안정을 위해 일정 금액이 지원됩니다.

2. **고용유지비용 지원**: 국가는 사업주가 근로자에게 육아휴직이나 육아기 근로시간 단축을 허용한 경우, 그 근로자의 생계비용과 사업주의 고용유지비용의 일부를 지원할 수 있습니다 (제20조 제1항).

3. **세제 및 재정 지원**: 국가는 소속 근로자의 일ㆍ가정의 양립을 지원하기 위해 육아기 재택근무 등 관련 제도 도입 시 세제 및 재정 지원을 할 수 있습니다 (제20조 제2항).

이 외에도 직장어린이집 설치 및 운영에 대한 지원이 있으나, 이는 육아휴직 자체와 직접 관련된 지원금이라기보다는 일·가정 양립을 지원하는 제도로 볼 수 있습니다 (제21조, 제21조의2). 

위 지원내용은 육아휴직을 사용하는 근로자 및 사업주가 받을 수 있는 혜택을 중심으로 언급된 것으로, 구체적인 신청 절차나 세부 사항은 관련 기관에 문의하시길 권장합니다.

육아휴직은 일반적으로 사업주가 허용하여야 하지만, 특정한 경우에는 이를 거절할 수 있습니다. 「남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률」 제19조에 따르면 육아휴직은 임신 중인 여성 근로자 또는 만 8세 이하 또는 초등학교 2학년 이하의 자녀를 양육하기 위하여 신청하는 경우에 허용해야 합니다. 다만, 대통령령으로 정하는 경우에는 예외가 있습니다. 여기서 명시된 '대통령령으로 정하는 경우'는 구체적으로 적혀있지 않으므로, 법과 관련된 시행령 등에서 더 자세한 기준이 제시될 수 있습니다. 따라서, 육아휴직을 반려하는 구체적인 기준은 관련 대통령령을 참조하시면 확인할 수 있을 것입니다. 

(참고: 제19조 제1항)

남녀고용평등법 자체는 4대보험에 대해 구체적으로 언급하고 있지 않습니다. 하지만 일반적으로 4대보험(국민연금, 

In [23]:
store

{'leeji0422': InMemoryChatMessageHistory(messages=[HumanMessage(content='입양을 한 경우에도 신청 가능할까요?', additional_kwargs={}, response_metadata={}), AIMessage(content='네, 입양한 자녀의 경우에도 육아휴직을 신청할 수 있습니다. [제19조(육아휴직) 제1항]에 따르면, 임신 중인 여성 근로자뿐만 아니라 만 8세 이하 또는 초등학교 2학년 이하의 자녀를 양육하기 위해서라면 육아휴직을 신청할 수 있으며, 입양한 자녀도 해당됩니다.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'service_tier': 'default'}, id='run--dd2c8392-443d-4b92-8de0-b52d29bb79c2-0'), HumanMessage(content='아이를 낳고 육아휴직 신청을 했는데, 그 사이에 아이를 하나 더 가질 경우 연속으로 육아휴직 신청 가능한가요?', additional_kwargs={}, response_metadata={}), AIMessage(content='네, 가능합니다. 육아휴직은 만 8세 이하 또는 초등학교 2학년 이하의 자녀를 양육하기 위하여 신청할 수 있으며, 각 자녀에 대해 육아휴직을 신청할 수 있습니다. 따라서 첫 아이의 육아휴직이 끝난 후 두 번째 아이에 대한 육아휴직을 연속적으로 신청할 수 있습니다. 이와 관련된 육아휴직 규정은 제19조에 명시되어 있습니다.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8',

In [24]:
%whos

Variable                         Type                          Data/Info
------------------------------------------------------------------------
BaseChatMessageHistory           ABCMeta                       <class 'langchain_core.ch<...>.BaseChatMessageHistory'>
ChatOpenAI                       ModelMetaclass                <class 'langchain_openai.<...>_models.base.ChatOpenAI'>
ChatPromptTemplate               ModelMetaclass                <class 'langchain_core.pr<...>chat.ChatPromptTemplate'>
InMemoryChatMessageHistory       ModelMetaclass                <class 'langchain_core.ch<...>emoryChatMessageHistory'>
RunnablePassthrough              ModelMetaclass                <class 'langchain_core.ru<...>ugh.RunnablePassthrough'>
RunnableWithMessageHistory       ModelMetaclass                <class 'langchain_core.ru<...>nableWithMessageHistory'>
StreamingStdOutCallbackHandler   type                          <class 'langchain_core.ca<...>ngStdOutCallbackHandler'>
chain                

In [25]:
store.keys()

dict_keys(['leeji0422'])

In [26]:
for message in get_history('leeji0422').messages:
    print(f'[{message.type.upper()}]: {message.content}\n')

[HUMAN]: 입양을 한 경우에도 신청 가능할까요?

[AI]: 네, 입양한 자녀의 경우에도 육아휴직을 신청할 수 있습니다. [제19조(육아휴직) 제1항]에 따르면, 임신 중인 여성 근로자뿐만 아니라 만 8세 이하 또는 초등학교 2학년 이하의 자녀를 양육하기 위해서라면 육아휴직을 신청할 수 있으며, 입양한 자녀도 해당됩니다.

[HUMAN]: 아이를 낳고 육아휴직 신청을 했는데, 그 사이에 아이를 하나 더 가질 경우 연속으로 육아휴직 신청 가능한가요?

[AI]: 네, 가능합니다. 육아휴직은 만 8세 이하 또는 초등학교 2학년 이하의 자녀를 양육하기 위하여 신청할 수 있으며, 각 자녀에 대해 육아휴직을 신청할 수 있습니다. 따라서 첫 아이의 육아휴직이 끝난 후 두 번째 아이에 대한 육아휴직을 연속적으로 신청할 수 있습니다. 이와 관련된 육아휴직 규정은 제19조에 명시되어 있습니다.

[HUMAN]: 육아휴직으로 인한 정부지원금 종류가 뭐뭐 있나요

[AI]: 남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률에 따르면, 육아휴직과 관련하여 국가에서 지원할 수 있는 주요 지원금은 다음과 같습니다.

1. **육아휴직 급여**: 고용보험에서 지원되는 급여로, 근로자가 육아휴직을 사용하는 경우 경력단절을 방지하고 생활 안정을 위해 일정 금액이 지원됩니다.

2. **고용유지비용 지원**: 국가는 사업주가 근로자에게 육아휴직이나 육아기 근로시간 단축을 허용한 경우, 그 근로자의 생계비용과 사업주의 고용유지비용의 일부를 지원할 수 있습니다 (제20조 제1항).

3. **세제 및 재정 지원**: 국가는 소속 근로자의 일ㆍ가정의 양립을 지원하기 위해 육아기 재택근무 등 관련 제도 도입 시 세제 및 재정 지원을 할 수 있습니다 (제20조 제2항).

이 외에도 직장어린이집 설치 및 운영에 대한 지원이 있으나, 이는 육아휴직 자체와 직접 관련된 지원금이라기보다는 일·가정 양립을 지원하는 제도로 볼 수 있습니다 (제21조, 제21조의2). 

위 지원내용은 육아휴직을 사용하는 근